In [11]:
from config import sparqlTerms, mig_ns, sparql_mig_test, sparql_mig_simple, sparql_mig_dev
from Titles import date_from_title
from Supplementary_Thesis_Materials import Supplementary_Thesis_Materials
from BryanGruhn import BryanGruhn
from isVerOf import date_from_isVersionOf
from SPARQLWrapper import JSON, SPARQLWrapper
import re, os, json, requests
import nltk
#nltk.download('punkt')
from nltk import word_tokenize

In [ ]:
class Query:
    def __init__(self, strlen, delim):
        self.len = strlen
        self.delim = delim
        self.sparql = SPARQLWrapper(sparql_mig_dev)
        

    def getTypebyLen(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        self.out = []
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?type (count(?type) as ?count) (sample(?s) as ?resource) where {?s dcterm:type ?type {select ?s ?value WHERE {?s dcterm:created ?value. filter(contains(str(?value), str('%s')) && STRLEN(str(?value)) = %s)}}} Group by ?type" % (self.delim, self.len)
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            if triple['type']['value'] != '':
                self.out.append(triple['type']['value'] + " \t " + triple['count']['value'] + "\t" + triple['resource']['value'] + "\n") 
                
    def getDateLen(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        self.out = []
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?value (count(?value) as ?count) (sample(?s) as ?resource) WHERE {?s dcterm:created ?value. filter(contains(str(?value), str('%s')) && (STRLEN(str(?value)) = %s))} GROUP BY ?value" % (self.delim, self.len)
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            if triple['value']['value'] != '':
                self.out.append(triple['value']['value'] + " \t " + triple['count']['value'] + "\t" + triple['resource']['value'] + "\n")    
                
    def SampleDateLen(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        self.out = []
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?value ?resource WHERE {?resource dcterm:created ?value. filter(contains(str(?value), str('%s')) && (STRLEN(str(?value)) = %s))} LIMIT 1" % (self.delim, self.len)
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            if triple['value']['value'] != '':
                self.out.append(triple['value']['value'] + " \t " + triple['resource']['value'] + "\n")  
                
    def isVersionOf(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?s ?value WHERE {?s dcterm:isVersionOf ?value. {select ?s where {?s dcterm:created ?v filter(?v='')}} filter(?value != '')}"
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            text = triple['value']['value']
            resource = triple['s']['value']
            #print (resource)
            tokens = word_tokenize(text)
            for n,i in enumerate(tokens):
                if i == "," :
                    del tokens[n]
                if i == ")" :
                    del tokens[n]
                if i == "(" :
                    del tokens[n]
            years = Word(tokens)
            yearday = years.getyday()
            if yearday == None:
                # if a yyyy/mm/dd was not found search for other types of dates
                yearmonth = years.getymonth()
                if yearmonth == None:
                    # if a yyyy/mm was not found search for other types of dates
                    trans = years.getyear()
                    yyyy = Search(trans, resource)
                    yyyy.con()       
                    
    def BryanGruhn(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?s ?file where {?s dcterm:type ?type . ?s dcterm:created ?created . ?s ual:hasCollection ?collection . ?s <info:fedora/fedora-system:def/model#downloadFilename> ?file . filter(?created = '' && contains(?collection, 'Bryan/Gruhn'))}  order by ?type"
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        out = []
        with open('BryanGruhn.py', 'a') as p:
            p.write('BryanGruhn = [')
            for triple in r:
                text = triple['file']['value']
                resource = triple['s']['value']
                year = text.replace('.', ' ').replace('-', ' ').split(' ')
                if int(year[0]) > 1900 and int(year[0]) < 2018:
                    p.write('{"subject": ' + '"' + resource + '",' + "\n" + '"predicate": "http://purl.org/dc/terms/created"' + ',' + "\n" + '"object" :' + '"' + year[0] + '"' + "}," + "\n")
            p.writelines(']') 
            
    def supplThesisMat(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?s ?collection ?gradDate where {?s dcterm:type ?type . ?s dcterm:relation ?rel . ?s ual:hasCollection ?collection . {select ?rel ?gradDate where {?r <http://terms.library.ualberta.ca/id/fedora3handle> ?rel . ?r uald:graduationdate ?gradDate . filter(?gradDate != '' && ?rel != '')}} filter( contains(?collection, 'Supplementary'))}  "
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        out = []
        with open('Supplementary_Thesis_Materials.py', 'a') as p:
            p.write('Supplementary_Thesis_Materials = [')
            for triple in r:
                text = triple['gradDate']['value']
                resource = triple['s']['value']
                p.write('{"subject": ' + '"' + resource + '",' + "\n" + '"predicate": "http://purl.org/dc/terms/created"' + ',' + "\n" + '"object" :' + '"' + text + '"' + "}," + "\n")
            p.writelines(']') 
            
    def titleExtract(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?s ?collection ?title where {?s ual:hasCollection ?collection . ?s dcterm:title ?title . ?s dcterm:created ?created . filter(?created = '' && ?collection != 'Dr. Otto Schaefer Collection' && ?collection != 'Supplementary Thesis Materials' && ?collection != 'Theses and Dissertations' && ?collection != 'Bryan/Gruhn Archaeology Collection')} "
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            text = triple['title']['value']
            resource = triple['s']['value']
            print (resource)
            tokens = word_tokenize(text)
            for n,i in enumerate(tokens):
                if i == "," :
                    del tokens[n]
                if i == ")" :
                    del tokens[n]
                if i == "(" :
                    del tokens[n]
            years = Word(tokens)
            #yearday = years.getyday()
            #if yearday == None:
                # if a yyyy/mm/dd was not found search for other types of dates
             #   yearmonth = years.getymonth()
              #  if yearmonth == None:
                    # if a yyyy/mm was not found search for other types of dates
            trans = years.getyear()
            yyyy = Search(trans, resource)
            yyyy.con()
            
    def isVerOf(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?s ?collection ?ver ?title where {?s ual:hasCollection ?collection . ?s dcterm:title ?title . ?s dcterm:isVersionOf ?ver .  ?s dcterm:created ?created . filter(?ver != '' && ?created = '' && ?collection != 'Dr. Otto Schaefer Collection' && ?collection != 'Supplementary Thesis Materials' && ?collection != 'Theses and Dissertations' && ?collection != 'Bryan/Gruhn Archaeology Collection')}"
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            text = triple['ver']['value']
            resource = triple['s']['value']
            print (resource)
            tokens = word_tokenize(text)
            for n,i in enumerate(tokens):
                if i == "," :
                    del tokens[n]
                if i == ")" :
                    del tokens[n]
                if i == "(" :
                    del tokens[n]
            years = Word(tokens)
            #yearday = years.getyday()
            #if yearday == None:
                # if a yyyy/mm/dd was not found search for other types of dates
             #   yearmonth = years.getymonth()
              #  if yearmonth == None:
                    # if a yyyy/mm was not found search for other types of dates
            trans = years.getyear()
            yyyy = Search(trans, resource)
            yyyy.con()

In [ ]:
nltk.download('punkt'def main():
    for i in range(1, 31):
        for j in [";", ":", "-", "/", ".", ",", " "]:
            dat = Query(i, j)
            with open("Dates/len " + str(i) + " dates.tsv" , "a") as file:
                print ("len is: " + str(i), "Delimiter is: " + str(j))
                dd = dat.getDateLen()
                file.writelines(j + "\n")
                file.writelines(dat.out)
                file.writelines("-----------------------------------------------------------" + "\n")
            with open("Types/len " + str(i) + " types.tsv" , "a") as file:
                print ("len is: " + str(i), "Delimiter is: " + str(j))
                dd = dat.getTypebyLen()
                file.writelines(j + "\n")
                file.writelines(dat.out)
                file.writelines("-----------------------------------------------------------" + "\n")
            with open("Sample/len " + str(i) + " sample.tsv" , "a") as file:
                print ("len is: " + str(i), "Delimiter is: " + str(j))
                dd = dat.SampleDateLen()
                file.writelines(j + "\n")
                file.writelines(dat.out)
                file.writelines("-----------------------------------------------------------" + "\n")

In [ ]:
if __name__ == "__main__":
	main()

In [ ]:
class Word:
    def __init__(self, tokens):
        self.word = tokens
        self.out = []
        self.output = [""]
        
    def getyear(self):
        # find all years with a 4 digit format (between 1900 and 2018)
        self.year = [i for i, w in enumerate(self.word) if re.search('^\d{4}$', w)]
        #print (self.year)
        for n,year in enumerate(self.year):
            self.out.append({'year': '', "grams": {"1-gram": "", "-1-gram": "", "2-gram": "", "-2-gram": "", "3-gram": "", "-3-gram": "", "4-gram": "", "-4-gram": "", "5-gram": "", "-5-gram": ""}})
            if int(self.word[year]) > 1900 and int(self.word[year]) < 2018:
                #index = self.word.index(year)
                self.out[n]["year"] = self.word[year]
                for j in range(1, 6):
                    # preceding grams
                    if j + year < len(self.word): 
                        self.out[n]["grams"][str(j) + "-gram"] = self.word[year + j] 
                    #succeeding grams 
                    if year - j > 0:
                        self.out[n]["grams"]["-" + str(j) + "-gram"] = self.word[year - j]
                self.output = self.out 
        if self.output[0] != '':
            return self.output
    
    def getymonth(self):
        # find all dates with yyyy-./:mm format
        self.ymonth = [w for w in self.word if re.search('^\d{4}?[-/.//://\/]\d{2}$', w)]
        for i in self.ymonth:
            for j in ["-", ":", "/", "."]:
                if j in i:
                    self.year = ""
                    self.month = ""
                    self.year = i.split(str(j))[0]
                    self.month = i.split(str(j))[1]
                if self.year in range (1900, 2019) and self.month in range (1, 13):
                    self.output[0] = self.month + "/" + self.year
        if self.output[0] != '':
            return self.output
        else:
            return None
    
    def getyday(self):
        self.yday = [w for w in self.word if re.search('^\d{4}?[-/.//://\/]\d{2}?[/.//:/-/\/]\d{2}$', w)]
        for i in self.yday:
            for j in ["-", ":", "/", "."]:
                if j in i:
                    self.year = ""
                    self.month = ""
                    self.day = ""
                    self.year = i.split(str(j))[0]
                    self.month = i.split(str(j))[1]
                    self.day = i.split(str(j))[2]
                if self.year in range (1900, 2019) and self.month in range (1, 13) and slef.day in range (1, 32):
                    self.output[0] = self.day + "/" + self.month + "/" + self.year
        if self.output[0] != '':
            return self.output
        else:
            return None  
        

In [ ]:
class Search:
    def __init__(self, trans, resource):
        self.trans = trans
        self.resource = resource
        
    def con(self):
        # change year format to mm/dd/year
        with open("Titles.py", "a") as outfile:
            if type(self.trans) == list:
                outfile.write('{"subject" : ' + '"' + self.resource + '",')
                outfile.write("\n")
                outfile.write('"predicate" : ' + '"http://purl.org/dc/terms/created"' + ',')
                outfile.write("\n")
                self.date = []
                for n, i in enumerate(self.trans):
                    self.month = ""
                    self.day = ""
                    self.date.append(i["year"])
                    months = [{"month": 1, "mapping": ["Jan", "jan", "January", "january"]},
                                 {"month": 2, "mapping": ["Feb", "feb", "February", "february"]},
                                 {"month": 3, "mapping": ["Mar", "mar", "March", "march"]},
                                 {"month": 4, "mapping": ["Apr", "apr", "April", "april"]},
                                 {"month": 5, "mapping": ["May", "may", "05"]},
                                 {"month": 6, "mapping": ["Jun", "jun", "June", "june"]},
                                 {"month": 7, "mapping": ["Jul", "jul", "July", "july"]},
                                 {"month": 8, "mapping": ["Aug", "aug", "August", "august"]},
                                 {"month": 9, "mapping": ["Sep", "sep", "September", "september"]},
                                 {"month": 10, "mapping": ["Oct", "oct", "October", "october"]},
                                 {"month": 11, "mapping": ["Nov", "nov", "November", "november"]},
                                 {"month": 12, "mapping": ["Dec", "dec", "December", "december"]}]
                    days = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"]
                    for j in range(-5, 6):
                        # look through the grams for month
                        if j == 0:
                            continue   
                        for x in months:
                            self.month = i["grams"][str(j) + "-gram"].replace(".", "")
                            if self.month in x["mapping"]:
                                self.date[n] = (str(x["month"]) + "/" + i["year"])
                                # look for a day in the next gram
                                if j+1 != 0 and j+1<6:
                                    if i["grams"][str(j+1) + "-gram"] in days: 
                                        self.day = i["grams"][str(j+1) + "-gram"]                      
                                        self.date[n] = (str(x["month"]) + "/" +self.day + "/" + i["year"])
                #print (self.date)
                outfile.write('"object": ' + str(self.date))
                outfile.write("\n")
                outfile.write("}" + "," + "\n")

In [ ]:
dates = Query(1, 1)
extracts = dates.titleExtract()

In [ ]:
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
for i in [";", ":", "-", "/", ".", ",", " "]:
    query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT (count(?s) as ?count) WHERE {?s dcterm:created ?value. filter(contains(str(?value),  str('%s')))}" % (i)
    sparql.setQuery(query)
    r  = sparql.query().convert()['results']['bindings']
    for triple in r:
        if triple['count']['value'] != '':
            print (triple['count']['value'])

In [ ]:
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?s ?value WHERE {?s dcterm:isVersionOf ?value. {select ?s where {?s dcterm:created ?v filter(?v='')}} filter(?value != '')}"
sparql.setQuery(query)
r  = sparql.query().convert()['results']['bindings']
with open('triple.csv', 'w+') as p:
    for triple in r:
        p.write(triple['s']['value'] + " \t " + triple['value']['value'] + "\n")

In [ ]:
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?s ?type ?collection ?created where  {?s dcterm:type ?type . ?s ual:hasCollection ?collection . ?s dcterm:created ?created . filter(?type !='Thesis') }  order by (?created)"
sparql.setQuery(query)
r  = sparql.query().convert()['results']['bindings']
with open('Allcreated_nothesis.tsv', 'a') as p:
    for triple in r:
        p.write(triple['s']['value'] + " \t " + triple['type']['value'] + " \t " + triple['collection']['value'] + " \t " + triple['created']['value'] + " \n")

In [ ]:
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?s ?collection ?accDate where {?s dcterm:type ?type . ?s ual:hasCollection ?collection . ?s dcterm:created ?created . ?s uald:graduationdate ?gradDate . ?s dcterm:dateAccepted ?accDate filter(?gradDate = '' && ?created = '' && ?accDate != '' && ?type = 'Thesis')}  order by ?type"
sparql.setQuery(query)
r  = sparql.query().convert()['results']['bindings']
with open('Date_Accepted.tsv', 'a') as p:
    for triple in r:
        p.write(triple['s']['value'] + " \t " + triple['collection']['value'] + " \t " + "\t" + " \t " + triple['accDate']['value'] + " \n")

In [ ]:
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?s ?collection ?gradDate where {?s dcterm:type ?type . ?s dcterm:relation ?rel . ?s ual:hasCollection ?collection . {select ?rel ?gradDate where {?r <http://terms.library.ualberta.ca/id/fedora3handle> ?rel . ?r uald:graduationdate ?gradDate . filter(?gradDate != '' && ?rel != '')}} filter( contains(?collection, 'Supplementary'))}  "
sparql.setQuery(query)
r  = sparql.query().convert()['results']['bindings']
with open('Supplementray_thesis.tsv', 'a') as p:
    for triple in r:
        p.write(triple['s']['value'] + " \t " + triple['collection']['value'] + " \t " + triple['gradDate']['value'] + " \n")

In [2]:
for sub in date_from_title:
    subject = sub['subject']
    date = sub['object']
    sparql = SPARQLWrapper(sparql_mig_dev)
    sparql.setMethod("POST")
    sparql.setReturnFormat(JSON)
    query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?title ?collection where {<%s> ual:hasCollection ?collection . <%s> dcterm:title ?title}" % (subject, subject)
    sparql.setQuery(query)
    r  = sparql.query().convert()['results']['bindings']
    with open('date_from_title.tsv', 'a') as p:
        for triple in r:
            p.write(subject + " \t " + triple['collection']['value'] + " \t " + triple['title']['value'] + "\t" + str(date) + " \n")

In [ ]:
subject = sub['subject']
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?s ?collection ?ver ?title where {?s ual:hasCollection ?collection . ?s dcterm:title ?title . ?s dcterm:isVersionOf ?ver .  ?s dcterm:created ?created . filter(?ver != '' && ?created = '' && ?collection != 'Dr. Otto Schaefer Collection' && ?collection != 'Supplementary Thesis Materials' && ?collection != 'Theses and Dissertations' && ?collection != 'Bryan/Gruhn Archaeology Collection')}"
sparql.setQuery(query)
r  = sparql.query().convert()['results']['bindings']
    #with open('date_from_title.tsv', 'a') as p:
#for triple in r:
for sub in date_from_title:
    if subject in r['s']['value']:
        print (subject)
        #p.write(subject + " \t " + triple['collection']['value'] + " \t " + triple['title']['value'] + "\t" + str(date) + " \n")

In [10]:
for sub in date_from_isVersionOf:
    subject = sub['subject']
    date = sub['object']
    sparql = SPARQLWrapper(sparql_mig_dev)
    sparql.setMethod("POST")
    sparql.setReturnFormat(JSON)
    query = "prefix uald: <http://terms.library.ualberta.ca/date/> prefix dcterm: <http://purl.org/dc/terms/> prefix ual: <http://terms.library.ualberta.ca/identifiers/> select ?collection ?ver where {<%s> ual:hasCollection ?collection . <%s> dcterm:isVersionOf ?ver}" % (subject, subject)
    sparql.setQuery(query)
    r  = sparql.query().convert()['results']['bindings']
    with open('date_from_isVer.tsv', 'a') as p:
        for triple in r:
            p.write(subject + " \t " + triple['collection']['value'] + " \t " + triple['ver']['value'].replace("\n", ' ') + "\t" + str(date) + " \n")

In [26]:
dates = [{"subject" : "", "predicate" : "", "object" : ""}]
subjects =[]
for subs in BryanGruhn:
    dates.append(subs)
    subjects.append(subs["subject"])
for subs in date_from_title:
    subject = subs['subject']
    if subject in subjects:
        continue
    else:
        dates.append(subs)
        subjects.append(subject)
for subs in date_from_isVersionOf:
    subject = subs['subject']
    if subject in subjects:
        continue
    else:
        dates.append(subs)
        subjects.append(subject)
del dates[0]
with open("dates.py", "a") as output:
    output.write("dates = [")
    for subs in dates:
        output.write(str(subs))
        output.write(",")
    output.write("]")
        
        
            